In [1]:
# 1
import mysql.connector
import pandas as pd
import getpass 
password = getpass.getpass()
bd = "sakila"
cnx = mysql.connector.connect(user = "root",
                              password = password,
                              host = "localhost",
                              database = bd)

cnx.is_connected()
engine = cnx.cursor()

In [2]:
# 2
def rentals_month(engine, month, year):

    query = (f"""
    SELECT *
    FROM sakila.rental
    WHERE MONTH(sakila.rental.rental_date) = {month}
    AND YEAR(sakila.rental.rental_date) = {year}
    """)

    engine.execute(query)
    df_columns = [head[0] for head in engine.description]
    df = pd.DataFrame(engine.fetchall(), columns=df_columns)
    return df
month = input("Please insert the month you want to analyse: ")
year = input("Please insert the year you want to analyse: ")
df = rentals_month(engine,month,year)
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2725,506,2005-06-10 01:26:44,2,2006-02-15 21:30:53
1152,1154,2005-05-31 21:42:09,2732,59,2005-06-08 16:40:09,1,2006-02-15 21:30:53
1153,1155,2005-05-31 22:17:11,2048,251,2005-06-04 20:27:11,2,2006-02-15 21:30:53
1154,1156,2005-05-31 22:37:34,460,106,2005-06-01 23:02:34,2,2006-02-15 21:30:53


In [3]:
# 3
def rental_count_month (df, month,year):
    df2 = df.groupby("customer_id").agg({"rental_id":"count"})
    df2= df2.rename(columns={"rental_id":f"rentals_{month}_{year}"})
    return df2
df2 = rental_count_month (df, month,year)
df2

,rentals_5_2005
customer_id,
1,2
2,1
3,2
5,3
6,3
...,...
594,4
595,1
596,6


In [4]:
# 4
def compare_rentals ():
    month = 5
    month2 = 6
    df = rentals_month(engine,month,year)
    df2 = rentals_month(engine,month2,year)
    df3 = rental_count_month (df, month,year)
    df4 = rental_count_month (df2, month2,year)
    df5 = pd.merge(df3,df4, how="inner", on="customer_id")
    df5["difference"] = df5["rentals_5_2005"]-df5["rentals_6_2005"]
    return df5
df6 = compare_rentals()
df6

,rentals_5_2005,rentals_6_2005,difference
customer_id,,,
1,2,7,-5
2,1,1,0
3,2,4,-2
5,3,5,-2
6,3,4,-1
...,...,...,...
594,4,6,-2
595,1,2,-1
596,6,2,4
